In [2]:
from util.mesh import MeshProcessor
from tqdm import tqdm
import numpy as np
import dask
from dask.distributed import Client, progress
import pandas as pd
import socket


cell = "jrc_hela-2"

client = Client(threads_per_worker=2, n_workers=1)
client.cluster.scale(20)
dashboard_link = client.cluster.dashboard_link
print(dashboard_link.replace("127.0.0.1",socket.gethostbyname(socket.gethostname())))

mp = MeshProcessor(
    path=f"https://janelia-cosem-datasets.s3.amazonaws.com/{cell}/neuroglancer/mesh/mito_seg",
    lod=2,
    min_branch_length=100,
    use_skeletons=True,
)
lazy_results = []
for i in range(1, 422, 1):  # range(1,87077,100): #
    lazy_results.append(mp.process_mesh(id=i))
results = dask.compute(*lazy_results)
client.close()
df = pd.DataFrame.from_records(results)
temp = df[["pic_0", "pic_1", "pic_2"]].to_numpy() >= 0

http://10.150.100.248:8787/status


Warning at src/paths/unweighted.c:376 : Couldn't reach some vertices
Warning at src/paths/unweighted.c:376 : Couldn't reach some vertices
Warning at src/paths/unweighted.c:376 : Couldn't reach some vertices
Warning at src/paths/unweighted.c:376 : Couldn't reach some vertices
Warning at src/paths/unweighted.c:376 : Couldn't reach some vertices
Warning at src/paths/unweighted.c:376 : Couldn't reach some vertices
Warning at src/paths/unweighted.c:376 : Couldn't reach some vertices
Warning at src/paths/unweighted.c:376 : Couldn't reach some vertices
Warning at src/paths/unweighted.c:376 : Couldn't reach some vertices
Warning at src/paths/unweighted.c:376 : Couldn't reach some vertices
Warning at src/paths/unweighted.c:376 : Couldn't reach some vertices
Warning at src/paths/unweighted.c:376 : Couldn't reach some vertices
Warning at src/paths/unweighted.c:376 : Couldn't reach some vertices
Warning at src/paths/unweighted.c:376 : Couldn't reach some vertices
Warning at src/paths/unweighted.c:

In [24]:
from imp import reload
import util.neuroglancer_predictor as NP
import util.fit_and_predict as FP
np = NP.NeuroglancerPredictor("jrc_hela-2")
np.setup_neuroglancer()
fp = FP.FitAndPredict(df, np)
fp.set_metrics(["volume"])

In [25]:
fp.set_metrics(
        [
            "ob_normalized_0",
            "ob_normalized_1",
            "ob_normalized_2",
            "pic_normalized_0",
            "pic_normalized_1",
            "pic_normalized_2",
            "longest_path",
            "num_fragments",
        ]   
)
